In [1]:
from pathlib import Path
import os
workdir = Path("/ibstorage/anthony/NYS_Wetlands_DL/")
print(workdir)
os.chdir(workdir)
current_working_dir = Path.cwd()
print(f"Current working directory is now: {current_working_dir}")

/ibstorage/anthony/NYS_Wetlands_DL
Current working directory is now: /ibstorage/anthony/NYS_Wetlands_DL


In [2]:
import ee
import geemap
import geemap.foliumap as geemap
import geopandas as gpd
import ipywidgets as widgets
import os
import time
from google.cloud import storage

ee.Authenticate()

ee.Initialize()
ee.ServiceAccountCredentials(email = "ajs-gee-access@earthengineajs.iam.gserviceaccount.com", 
                              key_file="Python_Code_Analysis/API_Key/earthengineajs-ajs_gee_access.json")

# Set up authentication for Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "Python_Code_Analysis/API_Key/earthengineajs-ajs_gee_access.json"

storage_client = storage.Client()
bucket_name = "ajs_gee_data"
bucket = storage_client.bucket(bucket_name)
bucket

<Bucket: ajs_gee_data>

In [3]:
# Filter for New York state
ny = ee.FeatureCollection("TIGER/2018/States").filter(
    ee.Filter.eq("NAME", "New York")
)

# HUC12 watersheds within NY, excluding 
    # Doesn't seem to match HUC with WBD downloaded from USGS
# ny_hucs = (
#     ee.FeatureCollection("USGS/WBD/2017/HUC12")
#     .filterBounds(ny)
#     .filter(
#         ee.Filter.And(
#             ee.Filter.neq("name", "Lake Ontario"),
#             ee.Filter.neq("name", "Lake Erie"),
#             ee.Filter.neq("name", "Lacarpe Creek-Frontal Lake Erie"),
#             ee.Filter.neq("name", "041201040000"),
#             ee.Filter.neq("name", "020302030000"),
#             ee.Filter.neq("name", "Amherst Island-Frontal Lake Ontario"),
#             ee.Filter.neq("name", "041503100000"),
#             ee.Filter.neq("name", "041504090000"),
#             ee.Filter.neq("name", "041503090000"),
#             ee.Filter.neq("name", "Long Island Sound Deep"),
#         )
#     )
# )

ny_hucs = ee.FeatureCollection("projects/earthengineajs/assets/NY_Cluster_Zones_250_NAomit")

single_huc = ee.Feature(ny_hucs.first()).geometry()

# Read HUC IDs from your text file
with open('Data/Dataframes/HUCs_in_site_clusters_NAomit.txt', 'r') as f:
    huc_ids_of_interest = [line.strip() for line in f if line.strip()]

# Subset ny_hucs to only include those HUC IDs
ny_hucs_clusters = ny_hucs.filter(ee.Filter.inList('huc12', huc_ids_of_interest))

# Verify the count
print(f"Total HUCs of interest: {len(huc_ids_of_interest)}")
print(f"Matched HUCs in ny_hucs: {ny_hucs_clusters.size().getInfo()}")

Total HUCs of interest: 225
Matched HUCs in ny_hucs: 225


In [4]:
ny_hucs_clusters.aggregate_array('name').getInfo()

['Cranberry Lake-Oswegatchie River',
 'Buck Brook-Oswegatchie River',
 'Lower Little River',
 'Tamarack Creek',
 'Peavine Creek-Oswegatchie River',
 'Upper Little River',
 'Robinson River-Oswegatchie River',
 'Ausable River',
 'Outlet Little Ausable River',
 'Lake Champlain',
 'Dead Creek',
 'Saranac River',
 'Headwaters Salmon River',
 'Outlet Salmon River',
 'Kelly Brook-Saranac River',
 'Beaver Pond Brook-Oak Brook',
 'Outardes River',
 'Hinchinbrook Brook',
 'Marble River',
 'Collins Brook',
 'Little Trout River',
 'Town of Trout River-Trout River',
 'Beaver Pond Creek-Chateaugay River',
 'Bailey Brook-Chateaugay River',
 'East Outardes River',
 'Mitchel Creek',
 'Osgood River',
 'Duane Stream',
 'Ingraham Stream-Salmon River',
 'Hatch Brook',
 'Sumner Brook',
 'Alder Brook',
 'Middle North Branch Saranac River',
 'Hays Brook',
 'Mile Brook-Deer River',
 'Upper North Branch Saranac River',
 'Upper Otselic River',
 'Crooked Brook-Pleasant Brook',
 'South Lebanon Brook-Cold Spring Br

In [5]:
# Get all HUC IDs from ny_hucs as a Python list
ny_hucs_all_ids = ny_hucs.aggregate_array('huc12').getInfo()

# Find HUC IDs that are in your file but not in ny_hucs
unmatched_hucs = [huc_id for huc_id in huc_ids_of_interest if huc_id not in ny_hucs_all_ids]

print(f"Unmatched HUCs: {len(unmatched_hucs)}")
print(unmatched_hucs)

Unmatched HUCs: 0
[]


In [6]:
# Define Image ImageCollection
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
naip = ee.ImageCollection("USDA/NAIP/DOQQ")

# Define time period
start_date = '2020-01-01'
end_date = '2025-10-01'

# Define start and end DOY
start_doy = 152  # Start of June
end_doy = 273  # end of Sept


def mask_s2_clouds(image):
    """Mask clouds and cloud shadows in Sentinel-2 imagery using SCL band."""
    scl = image.select('SCL')
    # Keep vegetation, bare soils, water, snow (classes 4, 5, 6, 11)
    # Exclude clouds (7, 8, 9, 10) and shadows (3)
    mask = scl.neq(3).And(scl.neq(7)).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))
    return image.updateMask(mask).divide(10000)


def calculate_SAR_indices(image):
    """Calculate SAR-based indices from Sentinel-1 data."""
    VV = image.select("VV")
    VH = image.select("VH")

    # Dual-Polarization SAR Vegetation Index
    DPSVI = VV.subtract(VH).divide(VV.add(VH)).rename('DPSVI')
    
    # Radar Vegetation Index (RVI)
    RVI = VH.multiply(4).divide(VV.add(VH)).rename('RVI')
    
    # VH/VV ratio (useful for vegetation/water discrimination)
    VH_VV_ratio = VH.divide(VV).rename('VH_VV_ratio')
    
    return image.addBands([DPSVI, RVI, VH_VV_ratio])


def calculate_indices(image):
    """Calculate spectral indices from Sentinel-2 data."""
    # Extract bands
    NIR = image.select('B8')
    RED = image.select('B4')
    REDG2 = image.select("B6")
    GREEN = image.select('B3')
    BLUE = image.select('B2')
    SWIR1 = image.select('B11')

    # Calculate spectral indices
    NDVI = (NIR.subtract(RED)).divide(NIR.add(RED)).rename('NDVI')
    MNDWI = (GREEN.subtract(SWIR1)).divide(GREEN.add(SWIR1)).rename('MNDWI')
    PSRI = (RED.subtract(BLUE)).divide(REDG2).rename("PSRI")  # Plant senescence 
    NDYI = (GREEN.subtract(BLUE)).divide(GREEN.add(BLUE)).rename('NDYI')  # Yellow
    GDVI = (NIR.subtract(GREEN)).divide(NIR.add(GREEN)).rename('GDVI')
    
    # Calculate EVI (Sentinel2)
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {'NIR': NIR, 'RED': RED, 'BLUE': BLUE}
    ).rename('EVI')

    # Add bands to the image
    image = image.addBands([NDVI, MNDWI, EVI, NDYI, PSRI, GDVI]).float()

    return image


def get_s2_composite(geometry, start_date, end_date, start_doy, end_doy):
    """Get Sentinel-2 composite with indices for a given geometry."""
    s2_filtered = (
        s2.filterBounds(geometry)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.dayOfYear(start_doy, end_doy))
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(mask_s2_clouds)
        .map(calculate_indices)
    )
    
    # Select only the index bands for the composite
    index_bands = ['NDVI', 'MNDWI', 'EVI', 'NDYI', 'PSRI', 'GDVI']
    
    # Create median composite of indices
    composite = s2_filtered.select(index_bands).median().clip(geometry)
    
    return composite


def get_s1_composite(geometry, start_date, end_date, start_doy, end_doy):
    """Get Sentinel-1 composite with indices for a given geometry."""
    s1_filtered = (
        s1.filterBounds(geometry)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.dayOfYear(start_doy, end_doy))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .map(calculate_SAR_indices)
    )
    
    # Select SAR index bands
    sar_bands = ['VV', 'VH', 'DPSVI', 'RVI', 'VH_VV_ratio']
    
    # Create median composite
    composite = s1_filtered.select(sar_bands).median().clip(geometry)
    
    return composite

In [7]:
def export_huc_indices(huc_feature, folder_prefix="ny_huc_indices"):
    """
    Export Sentinel-1 and Sentinel-2 indices for a single HUC watershed.
    
    Args:
        huc_feature: ee.Feature representing the HUC watershed
        folder_prefix: Folder name prefix in the GCS bucket
    
    Returns:
        Dictionary containing export task information
    """
    huc_id = huc_feature.get('huc12').getInfo()
    huc_name = huc_feature.get('name').getInfo()
    geometry = huc_feature.geometry()
    
    # Sanitize name for file naming
    safe_name = huc_name.replace(' ', '_').replace('/', '_').replace('\\', '_')
    
    # Get composites
    s2_composite = get_s2_composite(geometry, start_date, end_date, start_doy, end_doy)
    s1_composite = get_s1_composite(geometry, start_date, end_date, start_doy, end_doy)
    
    # Combine S1 and S2 indices into single image and cast to Float32
    combined_composite = s2_composite.addBands(s1_composite).toFloat()
    
    # Export configuration
    export_params = {
        'scale': 10,
        'crs': 'EPSG:4326',
        'maxPixels': 1e13,
        'fileFormat': 'GeoTIFF',
    }
    
    # Export combined indices
    task = ee.batch.Export.image.toCloudStorage(
        image=combined_composite,
        description=f'HUC_{huc_id}_indices',
        bucket=bucket_name,
        fileNamePrefix=f'{folder_prefix}/{huc_id}_{safe_name}_indices',
        region=geometry,
        **export_params
    )
    task.start()
    
    return {
        'huc_id': huc_id,
        'huc_name': huc_name,
        'task_id': task.id,
        'status': task.status()
    }

def export_all_hucs(huc_collection=None, batch_size=10, delay_seconds=1):
    """
    Export indices for all HUC watersheds in the provided collection.
    
    Args:
        huc_collection: ee.FeatureCollection of HUCs to export. Defaults to ny_hucs.
        batch_size: Number of exports to submit before printing status
        delay_seconds: Delay between task submissions to avoid rate limiting
    
    Returns:
        List of export task information dictionaries
    """
    # if huc_collection is None:
    #     huc_collection = ny_hucs
    
    # Get list of HUC features
    huc_list = huc_collection.toList(huc_collection.size())
    num_hucs = huc_collection.size().getInfo()
    
    print(f"Processing {num_hucs} HUC watersheds...")
    print(f"Exporting to gs://{bucket_name}")
    
    tasks = []
    
    for i in range(num_hucs):
        huc_feature = ee.Feature(huc_list.get(i))
        
        try:
            task_info = export_huc_indices(huc_feature)
            tasks.append(task_info)
            print(f"[{i+1}/{num_hucs}] Started export for HUC {task_info['huc_id']}: {task_info['huc_name']}")
            
            time.sleep(delay_seconds)
            
        except Exception as e:
            print(f"[{i+1}/{num_hucs}] Error processing HUC: {str(e)}")
            tasks.append({'error': str(e), 'index': i})
        
        if (i + 1) % batch_size == 0:
            print(f"\nBatch complete: {i+1}/{num_hucs} tasks submitted\n")
    
    print(f"\n{len(tasks)} export tasks submitted.")
    print("Check progress at: https://code.earthengine.google.com/tasks")
    
    return tasks


def check_task_status(tasks):
    """Check and print status of all export tasks."""
    status_counts = {'COMPLETED': 0, 'RUNNING': 0, 'PENDING': 0, 'FAILED': 0, 'CANCELLED': 0}
    
    for task_info in tasks:
        if 'error' in task_info:
            status_counts['FAILED'] += 1
            continue
            
        try:
            task = ee.batch.Task.list()[0]  # This is a simplification
            # In practice, you'd need to track tasks by ID
            status = ee.data.getTaskStatus(task_info['task_id'])[0]['state']
            status_counts[status] = status_counts.get(status, 0) + 1
        except:
            pass
    
    print("\nTask Status Summary:")
    for status, count in status_counts.items():
        if count > 0:
            print(f"  {status}: {count}")




In [23]:
s2_single_huc = get_s2_composite(geometry=ny_hucs.first().geometry(), start_date=start_date,
                                 end_date=end_date, 
                                 start_doy=start_doy,
                                 end_doy=end_doy
                                )
(s2_single_huc.bandNames())

Map = geemap.Map()
Map.add_basemap("HYBRID")
Map.center_object(ny_hucs.first())
Map.addLayer(s2_single_huc, {"bands":"EVI", "min":0, "max": 1, 'palette': ['brown', 'yellow', 'green']}, "EVI")
Map.add_colorbar(
    vis_params={'min': 0, 'max': 1, 'palette': ['brown', 'yellow', 'green']},
    label='EVI',
    position='bottomright'
)
Map


In [24]:
geemap.get_info(s2_single_huc, opened=True)

Tree(nodes=(Node(name='Image (6 bands)', nodes=(Node(icon='file', name='type: Image'), Node(name='bands: List …

In [64]:
def run_demo(n_hucs=3):
    """Test export on a small number of HUCs before full export."""
    # Get a limited list of HUCs
    huc_list = ny_hucs.toList(n_hucs)
    
    print(f"Testing export on {n_hucs} HUCs...")
    tasks = []
    
    for i in range(n_hucs):
        huc_feature = ee.Feature(huc_list.get(i))
        
        try:
            task_info = export_huc_indices(huc_feature, folder_prefix="ny_huc_indices_test")
            tasks.append(task_info)
            print(f"[{i+1}/{n_hucs}] Started: {task_info['huc_name']} ({task_info['huc_id']})")
        except Exception as e:
            print(f"[{i+1}/{n_hucs}] Error: {str(e)}")
    
    print(f"\nDone. {len(tasks)} tasks submitted.")
    print("Check progress at: https://code.earthengine.google.com/tasks")
    return tasks

# Test on 3 HUCs
# test_tasks = run_demo(3)


### Running HUC metric export for clusters

In [25]:
tasks = export_all_hucs(huc_collection=ny_hucs_clusters)

Processing 225 HUC watersheds...
Exporting to gs://ajs_gee_data
[1/225] Started export for HUC 042900030103: Cranberry Lake-Oswegatchie River
[2/225] Started export for HUC 042900030102: Buck Brook-Oswegatchie River
[3/225] Started export for HUC 042900030203: Lower Little River
[4/225] Started export for HUC 042900030201: Tamarack Creek
[5/225] Started export for HUC 042900030601: Peavine Creek-Oswegatchie River
[6/225] Started export for HUC 042900030202: Upper Little River
[7/225] Started export for HUC 042900030101: Robinson River-Oswegatchie River
[8/225] Started export for HUC 043001040302: Ausable River
[9/225] Started export for HUC 043001081302: Outlet Little Ausable River
[10/225] Started export for HUC 043001081604: Lake Champlain

Batch complete: 10/225 tasks submitted

[11/225] Started export for HUC 043001081602: Dead Creek
[12/225] Started export for HUC 043001060504: Saranac River
[13/225] Started export for HUC 043001081401: Headwaters Salmon River
[14/225] Started exp